In [1]:
import json
import pickle
from collections import defaultdict
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import matplotlib.patches as patches
from tqdm import tqdm
import os
import os, sys
sys.path.append("./Detectron_pytorch/lib")#path to detectron's lib
import numpy as np
from functools import reduce
from utils.boxes import bbox_overlaps

sys.path.append("./lib")#path to reldm's lib
from utils_rel.boxes_rel import boxes_union

### the following evaluation is adapted from:
https://github.com/NVIDIA/ContrastiveLosses4VRD/blob/master/lib/datasets_rel/task_evaluation_vg_and_vrd.py

In [2]:
#load predicted relations(attributes_with) on Test including gt
with open("./Outputs/imat_attr_test_gt_step499999/rel_detections.pkl","rb") as f:
    predicted_rel_att_with_output = pickle.load(f)
    
with open("./Outputs/imat_only_attr_step499999/rel_detections.pkl","rb") as f:
    predicted_rel_only_att_output = pickle.load(f)
    
with open("./Outputs/imat_only_with_step9999/rel_detections.pkl","rb") as f:
    predicted_rel_only_with_output = pickle.load(f)

with open("./Outputs/imat_attr_parent_test/rel_detections.pkl","rb") as f:
    predicted_rel_attr_parent_output = pickle.load(f)
    
    
with open('./data/imaterialist/predicates_attributes.json') as f:
    prd_cats = json.load(f)
with open('./data/imaterialist2/test_detections.json', 'rb') as handle:
    categories = json.load(handle)
    categories = categories["categories"]
    categories_id_to_name = {cat["id"]:cat["name"] for cat in categories}

In [3]:

def intersect_2d(x1, x2):
    """
    This function is adapted from Rowan Zellers' code: https://github.com/rowanz/neural-motifs
    Given two arrays [m1, n], [m2,n], returns a [m1, m2] array where each entry is True if those
    rows match.
    :param x1: [m1, n] numpy array
    :param x2: [m2, n] numpy array
    :return: [m1, m2] bool array of the intersections
    """
    if x1.shape[1] != x2.shape[1]:
        raise ValueError("Input arrays must have same #columns")

    # This performs a matrix multiplication-esque thing between the two arrays
    # Instead of summing, we want the equality, so we reduce in that way
    res = (x1[..., None] == x2.T[None, ...]).all(1)
    return res

def argsort_desc(scores):
    """
    Returns the indices that sort scores descending in a smart way
    :param scores: Numpy array of arbitrary size
    :return: an array of size [numel(scores), dim(scores)] where each row is the index you'd
             need to get the score.
    """
    return np.column_stack(np.unravel_index(np.argsort(-scores.ravel()), scores.shape))

# This function is adapted from Rowan Zellers' code:
# https://github.com/rowanz/neural-motifs/blob/master/lib/evaluation/sg_eval.py
# Modified for this project to work with PyTorch v0.4
def _compute_pred_matches(gt_triplets, pred_triplets,
                 gt_boxes, pred_boxes, iou_thresh=0.5, phrdet=False,experiment = "new_version"):
    """
    Given a set of predicted triplets, return the list of matching GT's for each of the
    given predictions
    :param gt_triplets: 
    :param pred_triplets: 
    :param gt_boxes: 
    :param pred_boxes: 
    :param iou_thresh: 
    :return: 
    """
    # This performs a matrix multiplication-esque thing between the two arrays
    # Instead of summing, we want the equality, so we reduce in that way
    # The rows correspond to GT triplets, columns to pred triplets
    
    keeps = intersect_2d(gt_triplets, pred_triplets)
    gt_has_match = keeps.any(1)
    
    pred_to_gt = [[] for x in range(pred_boxes.shape[0])]
    for gt_ind, gt_box, keep_inds in zip(np.where(gt_has_match)[0],
                                         gt_boxes[gt_has_match],
                                         keeps[gt_has_match],
                                         ):
        boxes = pred_boxes[keep_inds]
        if phrdet:
            gt_box = gt_box.astype(dtype=np.float32, copy=False)
            boxes = boxes.astype(dtype=np.float32, copy=False)
            rel_iou = bbox_overlaps(gt_box[None, :], boxes)[0]

            inds = rel_iou >= iou_thresh
        else:
            gt_box = gt_box.astype(dtype=np.float32, copy=False)
            if experiment == "old_version": # shabnam: our old model stored in x1,y1,x2,y2 -> converting to x1,y1,w,h
                gt_box[2] = gt_box[2] + gt_box[0]
                gt_box[3] = gt_box[3] + gt_box[1]

                gt_box[6] = gt_box[6] + gt_box[4]
                gt_box[7] = gt_box[7] + gt_box[5]
            
            boxes = boxes.astype(dtype=np.float32, copy=False)
            sub_iou = bbox_overlaps(gt_box[None,:4], boxes[:, :4])[0]
            obj_iou = bbox_overlaps(gt_box[None,4:], boxes[:, 4:])[0]
            inds = (sub_iou >= iou_thresh) & (obj_iou >= iou_thresh)

        for i in np.where(keep_inds)[0][inds]:
            pred_to_gt[i].append(int(gt_ind))
    return pred_to_gt

def print_stats(recalls):
    # print('====================== ' + 'sgdet' + ' ============================')
    for k, v in recalls.items():
        print('R@%i: %.2f' % (k, 100 * v))


In [4]:
thr_obj = 0.2 
thr_with = 0.1
thr_attr = 0.02
phrdet =False #False: reldet, True: phrdet
title = ["attr+with", "attr+parent","only with","only attr"]
for i, all_results in enumerate([predicted_rel_att_with_output,predicted_rel_attr_parent_output,predicted_rel_only_with_output,predicted_rel_only_att_output]):
    print(title[i])

    print('thr_obj={}, thr_with={}, thr_attr={}'.format(thr_obj,thr_with,thr_attr))

    recalls = { 20: 0, 50: 0, 100: 0}#1:0, 5:0, 10:0 ,
    topk = max(list(recalls.keys())) #shabnam: to save memory and time in earlier steps

    all_gt_cnt = 0

    topk_dets = []
    for im_i, res in enumerate(tqdm(all_results)):

        # in oi_all_rel some images have no dets
        assert res['prd_scores'] is not None

        det_boxes_sbj = res['sbj_boxes']  # (#num_rel, 4)
        det_boxes_obj = res['obj_boxes']  # (#num_rel, 4)
        det_labels_sbj = res['sbj_labels']  # (#num_rel,)
        det_labels_obj = res['obj_labels']  # (#num_rel,)
        det_scores_sbj = res['sbj_scores']  # (#num_rel,)
        det_scores_obj = res['obj_scores']  # (#num_rel,)
        if 'prd_scores_ttl' in res:
            det_scores_prd = res['prd_scores_ttl'][:, 1:]
        else:
            #det_scores_prd is a matrix [num_data * num_prd]
            det_scores_prd = res['prd_scores'][:, 1:]#excluding background


        #filter det_scores_prd for prd=with by thr_with
        det_scores_prd[:,0][det_scores_prd[:,0] < thr_with] = 0 
        #filter det_scores_prd for rest of prds by thr_attr
        det_scores_prd[det_scores_prd < thr_attr] = 0

        det_labels_prd = np.argsort(-det_scores_prd, axis=1)#sorting by predicats
        det_scores_prd = -np.sort(-det_scores_prd, axis=1)


        det_scores_so = det_scores_sbj * det_scores_obj
        #filter det_scores_so by thr_obj
        det_scores_so[det_scores_so < thr_obj] = 0
        det_scores_spo = det_scores_so[:, None] * det_scores_prd#[:, :prd_k] -> shabnam: reldn used prd_k to filter but we used thresholds

        det_scores_inds = argsort_desc(det_scores_spo)[:topk]
        det_scores_top = det_scores_spo[det_scores_inds[:, 0], det_scores_inds[:, 1]]
        det_boxes_so_top = np.hstack(
            (det_boxes_sbj[det_scores_inds[:, 0]], det_boxes_obj[det_scores_inds[:, 0]]))


        det_labels_p_top = det_labels_prd[det_scores_inds[:, 0], det_scores_inds[:, 1]]
        det_labels_spo_top = np.vstack(
            (det_labels_sbj[det_scores_inds[:, 0]], det_labels_p_top, det_labels_obj[det_scores_inds[:, 0]])).transpose()


        #shabnam: we trained attribute relations with format of object-attr-object(in case of attribute obj and subj can be the same)
        #therfore the second predicted obj does not matter and should affect our recall even if they are not correct
        #therefore for each predicted triplet subj-attr-object we manually add subj_attr_subj and obj_attr_obj to the predicted list
        det_boxes_so_top = np.append(det_boxes_so_top, np.hstack(
            (det_boxes_sbj[det_scores_inds[:, 0]], det_boxes_sbj[det_scores_inds[:, 0]])) ,0)
        det_boxes_so_top = np.append(det_boxes_so_top, np.hstack(
            (det_boxes_obj[det_scores_inds[:, 0]], det_boxes_obj[det_scores_inds[:, 0]])) ,0)
        det_labels_spo_top = np.append(det_labels_spo_top,np.vstack(
            (det_labels_sbj[det_scores_inds[:, 0]], det_labels_p_top, det_labels_sbj[det_scores_inds[:, 0]])).transpose(),0)
        det_labels_spo_top = np.append(det_labels_spo_top,np.vstack(
            (det_labels_obj[det_scores_inds[:, 0]], det_labels_p_top, det_labels_obj[det_scores_inds[:, 0]])).transpose(),0)
        #---------------------------------------------------------------


        det_boxes_s_top = det_boxes_so_top[:, :4]
        det_boxes_o_top = det_boxes_so_top[:, 4:]
        det_labels_s_top = det_labels_spo_top[:, 0]
        det_labels_p_top = det_labels_spo_top[:, 1]
        det_labels_o_top = det_labels_spo_top[:, 2]

        topk_dets.append(dict(image=res['image'],
                              det_boxes_s_top=det_boxes_s_top,
                              det_boxes_o_top=det_boxes_o_top,
                              det_labels_s_top=det_labels_s_top,
                              det_labels_p_top=det_labels_p_top,
                              det_labels_o_top=det_labels_o_top,
                              det_scores_top=det_scores_top))

        gt_boxes_sbj = res['gt_sbj_boxes']  # (#num_gt, 4)
        gt_boxes_obj = res['gt_obj_boxes']  # (#num_gt, 4)
        gt_labels_sbj = res['gt_sbj_labels']  # (#num_gt,)
        gt_labels_obj = res['gt_obj_labels']  # (#num_gt,)
        gt_labels_prd = res['gt_prd_labels']  # (#num_gt,)
        gt_boxes_so = np.hstack((gt_boxes_sbj, gt_boxes_obj))
        gt_labels_spo = np.vstack((gt_labels_sbj, gt_labels_prd, gt_labels_obj)).transpose()
        # Compute recall. It's most efficient to match once and then do recall after
        # det_boxes_so_top is (#num_rel, 8)
        # det_labels_spo_top is (#num_rel, 3)

        if phrdet:
            det_boxes_r_top = boxes_union(det_boxes_s_top, det_boxes_o_top)
            gt_boxes_r = boxes_union(gt_boxes_sbj, gt_boxes_obj)
            pred_to_gt = _compute_pred_matches(
                gt_labels_spo, det_labels_spo_top,
                gt_boxes_r, det_boxes_r_top,
                phrdet=phrdet)
        else:
            pred_to_gt = _compute_pred_matches(
                gt_labels_spo, det_labels_spo_top,
                gt_boxes_so, det_boxes_so_top,
                phrdet=phrdet, 
                experiment = "old_version" if title[i] == "attr+with" else "new_version")



        all_gt_cnt += gt_labels_spo.shape[0]
        for k in recalls:
            if len(pred_to_gt):
                breakpoint = len(pred_to_gt)//3
                match = reduce(np.union1d, pred_to_gt[:k])# exactly like reldn
                match = np.append(match ,reduce(np.union1d, pred_to_gt[breakpoint:breakpoint+k])) #for subj,attr,all_sbj
                match = np.append(match ,reduce(np.union1d, pred_to_gt[breakpoint+breakpoint:breakpoint+breakpoint+k]))#for obj,attr,all_obj
                match = list(set(match))


            else:
                match = []
            recalls[k] += len(match)

        topk_dets[-1].update(dict(gt_boxes_sbj=gt_boxes_sbj,
                                  gt_boxes_obj=gt_boxes_obj,
                                  gt_labels_sbj=gt_labels_sbj,
                                  gt_labels_obj=gt_labels_obj,
                                  gt_labels_prd=gt_labels_prd))
    for k in recalls:
        recalls[k] = float(recalls[k]) / (float(all_gt_cnt) + 1e-12)
    print_stats(recalls)

  0%|          | 14/9118 [00:00<01:08, 131.95it/s]

attr+with
thr_obj=0.2, thr_with=0.1, thr_attr=0.02


  0%|          | 13/9118 [00:00<01:10, 129.60it/s]

R@20: 18.34
R@50: 22.27
R@100: 24.98
attr+parent
thr_obj=0.2, thr_with=0.1, thr_attr=0.02


  0%|          | 21/8979 [00:00<00:44, 199.70it/s]

R@20: 24.78
R@50: 30.65
R@100: 34.09
only with
thr_obj=0.2, thr_with=0.1, thr_attr=0.02


  0%|          | 14/9118 [00:00<01:08, 132.61it/s]

R@20: 69.51
R@50: 70.93
R@100: 71.66
only attr
thr_obj=0.2, thr_with=0.1, thr_attr=0.02


100%|██████████| 9118/9118 [01:12<00:00, 125.72it/s]

R@20: 9.74
R@50: 12.59
R@100: 15.42
